In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["WANDB_DISABLED"] = "true"

import sys
sys.path = [
    p for p in sys.path
    if p not in ['/home/jxm3/research/prompting/imodelsX', '/home/jxm3/research/prompting/tree-prompt']
]
sys.path.append('/home/jxm3/research/retrieval/inversion')

In [8]:
import aliases

os.environ["TOKENIZERS_MULTIPROCESSING"] = "True"

# inv_trainer = aliases.load_trainer_from_alias("openai_msmarco__msl128__100epoch")
corr_experiment, corr_trainer = aliases.load_experiment_and_trainer_from_alias("gtr_nq__msl32_beta__correct")
inv_trainer = corr_trainer.inversion_trainer
# corr_trainer.precompute_hypotheses()
corr_trainer.model.eval()
print()

loading alias gtr_nq__msl32_beta__correct from /home/jxm3/research/retrieval/inversion/saves/47d9c149a8e827d0609abbeefdfd89ac...
> checkpoint: /home/jxm3/research/retrieval/inversion/saves/47d9c149a8e827d0609abbeefdfd89ac/checkpoint-558000
set dataset to nq
loading alias dpr_nq__msl32_beta from /home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea...
Set num workers to 4
Overwriting max sequence length from 32 to 32
Overwriting use_less_data from None to -1
> checkpoint: /home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea/checkpoint-999744


/home/jxm3/.conda/envs/torch/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Loading datasets with TOKENIZERS_PARALLELISM = True


[nltk_data] Downloading package wordnet to /home/jxm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jxm3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jxm3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Renaming keys {'embedding_transform.2.weight', 'embedding_transform.2.bias'} for backward compatibility.
================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.
	Decoded output shape ->  torch.Size([1, 33])
	Decoded output -> The mlbies wase wyst bograge; And the sliths and toms wy
================ End trainer sanity check ================
Froze 342572160 params from model type <class 'models.inversion.InversionModel'>


[nltk_data] Downloading package wordnet to /home/jxm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jxm3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jxm3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Renaming keys {'embedding_transform.2.weight', 'embedding_transform.2.bias'} for backward compatibility.
================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.
	Decoded output shape ->  torch.Size([1, 33])
	Decoded output -> The slithe and the tobogbes were mly; It wis grabbse tiring
================ End trainer sanity check ================



In [46]:
import datasets

# https://en.wikipedia.org/wiki/Mage_(horse)
text = "Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby."

# "Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.
# Mage is a chestnut colt bred in Kentucky by Grandview Equine. His sire Good Magic was the U.S. Champion Two-Year-Old in 2017 after he won the 2017 Breeders' Cup Juvenile as maiden; in 2018 he finished second in the Kentucky Derby.[4]


temp_dataset = datasets.Dataset.from_list([{"text": text}])
datasets_dict = datasets.DatasetDict({
    "mage": temp_dataset
})
d = corr_experiment._prepare_val_datasets_dict(
    model=corr_trainer.inversion_trainer.model,
    val_datasets_dict=datasets_dict,
    tokenizer=corr_trainer.tokenizer,
    embedder_tokenizer=corr_trainer.embedder_tokenizer,
)

Running tokenizer on dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1 [00:00<?, ? examples/s]

In [50]:
corr_trainer.gen_kwargs

{'early_stopping': False,
 'num_beams': 1,
 'do_sample': False,
 'no_repeat_ngram_size': 0}

In [64]:
dl = corr_trainer.get_eval_dataloader(d["mage"].select([0,]))
gen_kwargs = {
    "max_length": 32,
    **corr_trainer.gen_kwargs
}
corr_trainer.inversion_trainer.model.noise_level = .005
guess = corr_trainer.inversion_trainer.generate(
    inputs={k: v.cuda() for k,v in next(iter(dl)).items()},
    generation_kwargs=gen_kwargs
)
corr_trainer.tokenizer.batch_decode(guess, add_special_tokens=False)

['<pad> Kentucky Derby, which was won by Mage (April 20, 2010), who is an American Thoroughbred horse and mare. He made his final run']

In [31]:
corr_trainer.evaluate(eval_dataset=d["mage"].select([0, 0, 0]))

[pred] Mage (foaled April 20, 2020) is an American Thoroughbred racehorse who won the 1923 Kentucky Derby.
[true] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.



[pred] Mage (foaled April 20, 2020) is an American Thoroughbred racehorse who won the 1923 Kentucky Derby.
[true] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.



[pred] Mage (foaled April 20, 2020) is an American Thoroughbred racehorse who won the 1923 Kentucky Derby.
[true] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.


[pred] Mage (foaled April 20, 2020) is an American Thoroughbred racehorse who won the 1923 Kentucky Derby.
[true] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.



[pred] Mage (foaled April 20, 2020) is an American Thoroughbred racehorse who won the 1923 Kentucky Derby.
[true] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.



[pred] Mage (foaled April 20, 2020) is an American Thoroughbred racehorse who won the 1923 Kentucky Derby.
[true] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.


{'eval_loss': 0.13891343772411346,
 'eval_pred_num_tokens': 31.0,
 'eval_true_num_tokens': 32.0,
 'eval_token_set_precision': 0.9,
 'eval_token_set_recall': 0.9,
 'eval_token_set_f1': 0.9,
 'eval_n_ngrams_match_1': 18.0,
 'eval_n_ngrams_match_2': 15.0,
 'eval_n_ngrams_match_3': 12.0,
 'eval_num_true_words': 20.0,
 'eval_num_pred_words': 20.0,
 'eval_bleu_score': 70.76534431960266,
 'eval_meteor_score': 0.8979166666666667,
 'eval_rouge_score': 0.875,
 'eval_exact_match': 0.0,
 'eval_emb_cos_sim': 0.9964544773101807,
 'eval_runtime': 7.209,
 'eval_samples_per_second': 0.416,
 'eval_steps_per_second': 0.139}

In [34]:
corr_trainer.num_gen_recursive_steps = 2
corr_trainer.evaluate(eval_dataset=d["mage"].select([0, 0, 0]))

[pred] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.
[true] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.



[pred] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.
[true] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.



[pred] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.
[true] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.


[pred] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.
[true] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.



[pred] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.
[true] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.



[pred] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.
[true] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.


{'eval_loss': 0.13891343772411346,
 'eval_pred_num_tokens': 31.0,
 'eval_true_num_tokens': 32.0,
 'eval_token_set_precision': 1.0,
 'eval_token_set_recall': 1.0,
 'eval_token_set_f1': 1.0,
 'eval_n_ngrams_match_1': 20.0,
 'eval_n_ngrams_match_2': 19.0,
 'eval_n_ngrams_match_3': 18.0,
 'eval_num_true_words': 20.0,
 'eval_num_pred_words': 20.0,
 'eval_bleu_score': 100.00000000000004,
 'eval_meteor_score': 0.9999375,
 'eval_rouge_score': 1.0,
 'eval_exact_match': 1.0,
 'eval_emb_cos_sim': 1.0,
 'eval_runtime': 8.1186,
 'eval_samples_per_second': 0.37,
 'eval_steps_per_second': 0.123}

In [35]:
corr_trainer.num_gen_recursive_steps = 3
corr_trainer.evaluate(eval_dataset=d["mage"].select([0, 0, 0]))

[pred] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.
[true] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.



[pred] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.
[true] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.



[pred] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.
[true] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.


[pred] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.
[true] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.



[pred] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.
[true] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.



[pred] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.
[true] Mage (foaled April 18, 2020) is an American Thoroughbred racehorse who won the 2023 Kentucky Derby.


{'eval_loss': 0.13891343772411346,
 'eval_pred_num_tokens': 31.0,
 'eval_true_num_tokens': 32.0,
 'eval_token_set_precision': 1.0,
 'eval_token_set_recall': 1.0,
 'eval_token_set_f1': 1.0,
 'eval_n_ngrams_match_1': 20.0,
 'eval_n_ngrams_match_2': 19.0,
 'eval_n_ngrams_match_3': 18.0,
 'eval_num_true_words': 20.0,
 'eval_num_pred_words': 20.0,
 'eval_bleu_score': 100.00000000000004,
 'eval_meteor_score': 0.9999375,
 'eval_rouge_score': 1.0,
 'eval_exact_match': 1.0,
 'eval_emb_cos_sim': 1.0,
 'eval_runtime': 9.33,
 'eval_samples_per_second': 0.322,
 'eval_steps_per_second': 0.107}